In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
sample_submission = pd.read_csv("/kaggle/input/digit-recognizer/sample_submission.csv")
train = pd.read_csv("/kaggle/input/digit-recognizer/train.csv")
test = pd.read_csv("/kaggle/input/digit-recognizer/test.csv")

In [ ]:
images = train.values[:,1:].reshape(-1, 28, 28)[6]

In [ ]:
import tensorflow as tf

In [ ]:
inputs = tf.keras.Input(shape = (28, 28, 1))
    
C1 = tf.keras.layers.RandomContrast(0.2)(inputs)

M_1 = tf.keras.layers.Conv2D(16, 3, activation='relu', input_shape = (28, 28, 1), padding="same")(C1)
M_1 = tf.keras.layers.MaxPool2D(pool_size=(3, 3), strides=1, padding="same")(M_1)

M_2 = tf.keras.layers.Conv2D(8, 3, activation='relu', padding="same")(M_1)
M_2 = tf.keras.layers.MaxPool2D(pool_size=(3, 3), strides=2, padding="same")(M_2)

M_3 = tf.keras.layers.Conv2D(4, 3, activation='relu', padding="same")(M_2)
M_3 = tf.keras.layers.MaxPool2D(pool_size=(3, 3), strides=2, padding="valid")(M_3)

M = tf.keras.layers.Flatten()(M_3)
D1 = tf.keras.layers.Dense(16, activation="relu")(M)
D2 = tf.keras.layers.Dense(16, activation="relu")(D1)  

outputs = tf.keras.layers.Dense(10, activation="softmax", name="output")(D2)

model = tf.keras.Model(inputs=inputs, outputs=outputs, name="mnist_model")

model.compile(
    loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']
)

In [ ]:
for i in range(10):
    train[f'label_{i}'] = (train['label'] == i) * 1.0

In [ ]:
X = train[[c for c in train.columns if 'pixel' in c]].values.reshape(len(train), 28, 28, 1)
y = train[[c for c in train.columns if 'label_' in c]]

In [ ]:
model.fit(X, y, epochs = 20, validation_split = 0.2)

In [ ]:
model.summary()

In [ ]:
test['Label'] = model.predict(
    test[[c for c in test.columns if 'pixel' in c]].values.reshape(len(test), 28, 28, 1)
).argmax(axis = 1)

In [ ]:
test['ImageId'] = list(range(1, len(test) + 1))

In [ ]:
sample_submission.head()

In [ ]:
test.head()

In [ ]:
test.to_csv('submission.csv', index=False, columns = ['ImageId', 'Label'])